In [3]:
# !pip install lightautoml

In [31]:
import pandas as pd
import numpy as np
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, roc_curve, precision_score, recall_score, f1_score, precision_recall_curve, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/Данные/housing.csv"
df = pd.read_csv(path)

In [4]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [6]:
RANDOM_STATE = 123

In [19]:
# разбиваем на тренировочный сет и тестовый сет в отношении 0.8 / 0.2 соответственно

df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE)

In [20]:
# значения Y в тренировочном и тестовом сете

y_train = df_train['MEDV']
y_test = df_test['MEDV']

In [21]:
df_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,MEDV_prediction
203,0.03510,95.0,2.68,0.0,0.4161,7.853,33.2,5.1180,4.0,224.0,14.7,392.78,3.81,48.5,47.669228
441,9.72418,0.0,18.10,0.0,0.7400,6.406,97.2,2.0651,24.0,666.0,20.2,385.96,19.52,17.1,16.133839
172,0.13914,0.0,4.05,0.0,0.5100,5.572,88.5,2.5961,5.0,296.0,16.6,396.90,14.69,23.1,22.165575
95,0.12204,0.0,2.89,0.0,0.4450,6.625,57.8,3.4952,2.0,276.0,18.0,357.98,6.65,28.4,27.279957
54,0.01360,75.0,4.00,0.0,0.4100,5.888,47.6,7.3197,3.0,469.0,21.1,396.90,14.80,18.9,18.353722


In [22]:
task = Task(name='reg') # тип решаемой задачи - регрессия
roles = {'target': 'MEDV'} # целевой признак - 'MEDV'
automl = TabularAutoML(task=task, timeout=3600, memory_limit=10) # здесь создаем объект черной коробки LightAutoML
oof_pred = automl.fit_predict(df_train, roles=roles)

In [23]:
# удаляем target-столбец из тренировочного и тестового датасетов

df_train = df_train.drop('MEDV', axis=1)
df_test = df_test.drop('MEDV', axis=1)

In [24]:
# предсказываем значения Y для тренировочного и тестового датасетов

y_train_score = automl.predict(df_train).data
y_test_score = automl.predict(df_test).data

In [25]:
# предсказываем target-MEDV для датасета df

medv_prediction = automl.predict(df.drop('MEDV', axis=1)).data

In [26]:
df['MEDV_prediction'] = medv_prediction

In [27]:
df[['MEDV', 'MEDV_prediction']].head(10)

,MEDV,MEDV_prediction
0,24.0,24.253952
1,21.6,20.504570
2,34.7,35.335388
3,33.4,34.896667
4,36.2,35.387794
5,28.7,26.601152
6,22.9,22.176212
7,27.1,23.488592
8,16.5,16.324146
9,18.9,18.385231


In [33]:
print("RMSE тестовой выборки LinearRegression: ",np.sqrt(mean_squared_error(y_test, y_test_score)))

RMSE тестовой выборки LinearRegression:  3.7884544183017996
